In [13]:
        import sqlite3

conn = sqlite3.connect(r'../hate_speech_data.db')
cursor = conn.cursor()

In [16]:
from fuzzySearch import search_and_count_labels

In [18]:
search_and_count_labels(conn, 'hate')

dataset_id | count | label_names | label_definitions
4 | 82288 | label, type | a binary variable, indicating whether or not the content has been identified as hateful. It takes two values: hate, nothate., a categorical variable, providing a secondary label for hateful content. For hate it can take five values: Animosity, Derogation, Dehumanization, Threatening and Support for Hateful Entities. Please see the paper for more detail. For nothate the 'type' is 'none'. In round 1 the 'type' was not given and is marked as 'notgiven'
5 | 135556 | hate_speech_score | Continuous hate speech measure, where higher = more hateful and lower = less hateful. > 0.5 is approximately hate speech, < -1 is counter or supportive speech, and -1 to +0.5 is neutral or ambiguous 
6 | 3000 | HOF | if tweet text is hateful and offensive(Hateful) or neither hateful nor offensive(NonHateful)
9 | 998 | isHate | 
11 | 16625 | TARGET | cover 6 main targets of hate, namely JEWS, LGBT+, MIGRANTS, MUSLIMS, PEOPLE OF COL

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

### 4 DGHD generated N/A

In [24]:
query = """
SELECT text.text, label.label_value as label
FROM text
JOIN label ON label.text_id = text.text_id
WHERE text.dataset_id = 4 AND label.label_name = 'label';
"""
df = pd.read_sql_query(query, conn)

In [28]:
print(len(df))
print(df.describe())
print(df['label'].value_counts())
print(df.head(10))

70422
                            text  label
count                      70422  70422
unique                     41134      6
top     Blacks are such pondlife   hate
freq                           5  22175
label
hate       22175
nothate    18969
0           9776
1.0         7141
0.0         7141
1           5220
Name: count, dtype: int64
                                                text    label
0                                dalits are lowlives     hate
1             gay people are a burden to our society     hate
2                              Arabs are not welcome     hate
3  I'm not saying we should actually eliminate he...     hate
4                       bananas are for black people     hate
5                          asian people are the shit  nothate
6  women can not reproduce kids but men can in al...     hate
7                i hate wearing black in the summer!  nothate
8               Who cares what Chinese people think?     hate
9         i try to punch a letter box wh

In [33]:
df['label'] = df['label'].replace({'nothate': 0, '0.0':0, 'hate': 1, '1.0':1, '1':1, '0':0})

In [34]:
print(df['label'].value_counts())

label
0    35886
1    34536
Name: count, dtype: int64


In [35]:
df = df.drop_duplicates()

In [36]:
print(df['label'].value_counts())

label
1    26421
0    25592
Name: count, dtype: int64


In [39]:
df['dataset'] = 'DGHD'
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# train_df['set'] = 'train'
# test_df['set'] = 'test'
# df = pd.concat([train_df, test_df])

import os
os.makedirs('data', exist_ok=True)
file_path = os.path.join('data', 'DGHD.csv')
df.to_csv(file_path, index=False)

### 5 MHS Youtube, Twitter, Reddit

In [40]:
query = """
SELECT text.text, label.label_value as label
FROM text
JOIN label ON label.text_id = text.text_id
WHERE text.dataset_id = 5 AND label.label_name = 'hate_speech_score';
"""
df = pd.read_sql_query(query, conn)

In [42]:
print(len(df))
print(df.describe())
print(df.head(10))

135556
                                                     text   label
count                                              135556  135556
unique                                              39565    1255
top     JEW Get the fuck out of here you jewish son of...    2.73
freq                                                  815    1581
                                                text  label
0  Yes indeed. She sort of reminds me of the elde...   -3.9
1  The trans women reading this tweet right now i...  -6.52
2  Question: These 4 broads who criticize America...   0.36
3  It is about time for all illegals to go back t...   0.26
4  For starters bend over the one in pink and kic...   1.54
5  Sounds like the kinda wholsesome life I'd die ...  -4.93
6  Who the fuck is this insignificant simple mind...   0.17
7         Fuck off you insufferable retarded faggot.   2.08
8  OMG! The EGO's of these young, young, inexperi...  -0.66
9  Worthless whore, these tits with look nice wit...  -0.38


In [44]:
df['label'] = df['label'].astype(float).apply(lambda x: 1 if x > 0.5 else 0)

In [45]:
print(df['label'].value_counts())

label
0    86508
1    49048
Name: count, dtype: int64


In [46]:
df = df.drop_duplicates()

In [47]:
print(df['label'].value_counts())

label
0    29211
1    10354
Name: count, dtype: int64


In [48]:
df['dataset'] = 'MHS'

file_path = os.path.join('data', 'MHS.csv')
df.to_csv(file_path, index=False)

### 6 USElection2020 Twitter

In [49]:
query = """
SELECT text.text, label.label_value as label
FROM text
JOIN label ON label.text_id = text.text_id
WHERE text.dataset_id = 6 AND label.label_name = 'HOF';
"""
df = pd.read_sql_query(query, conn)

In [50]:
print(len(df))
print(df.describe())
print(df.head(10))

3000
                                                     text        label
count                                                3000         3000
unique                                               2999            2
top     The Democrat Party is in full meltdown! They l...  Non-Hateful
freq                                                    2         2648
                                                text        label
0  @SukiRavan @ProgressPotato @MarkZuckerb0rg @JS...  Non-Hateful
1  @Newsweek Are you freaking crazy????[NEWLINE]I...  Non-Hateful
2  Undecided voters (and MAGATs alike);[NEWLINE]I...  Non-Hateful
3  @cheaterwins @Hungry_For_More @DAYSORSHAY So a...  Non-Hateful
4  @CNN Nancy Pelosi and the Dems wont do a deal ...  Non-Hateful
5  @EvanAKilgore @PressSec @johnrobertsFox Looks ...  Non-Hateful
6  @ACTforAmerica @realDonaldTrump We love him to...  Non-Hateful
7  @realDonaldTrump I don't have a 401k #bidenhar...  Non-Hateful
8  @kayleighmcenany @realDonaldTrump Omfg....T

In [51]:
df['label'] = df['label'].replace({'Non-Hateful': 0, 'Hateful':1})

In [52]:
print(df['label'].value_counts())

label
0    2648
1     352
Name: count, dtype: int64


In [53]:
df = df.drop_duplicates()

In [54]:
print(df['label'].value_counts())

label
0    2648
1     351
Name: count, dtype: int64


In [55]:
df['dataset'] = 'USElection2020'

file_path = os.path.join('data', 'USElection2020.csv')
df.to_csv(file_path, index=False)

### 23 fox-comment

In [56]:
query = """
SELECT text.text, label.label_value as label
FROM text
JOIN label ON label.text_id = text.text_id
WHERE text.dataset_id = 23 AND label.label_name = 'hateful';
"""
df = pd.read_sql_query(query, conn)

In [57]:
print(len(df))
print(df.describe())
print(df.head(10))

5528
                                                     text label
count                                                5528  5528
unique                                               3994     2
top     Un inmigrante ghanés mata a un matrimonio en F...     0
freq                                                    6  4017
                                                text label
0  Avión venezolano: ocho tripulantes coinciden e...     0
1  Aquí pidiéndole consejos al amigo veneco pa so...     0
2  @oamz Cínicos desinforman con guión preparado,...     0
3                   @oamj No se chama dime tu 🤭🤭🤣😛😅😅     1
4  Gobierno de Bélgica ustedes que albergan a Raf...     1
5  A todas las personas que han tenido que movers...     0
6  Y esto es lo que pasa cuando dejas a auténtico...     0
7  Repudiamos los actos vandalicos y responsabili...     0
8  @ohuj @osap @oaiw @oarq @oaqm @oaix @ofgz @oxt...     1
9  @oxtr Eso me dijeron dos venezolanas en Miami,...     0


In [58]:
print(df['label'].value_counts())

label
0    4017
1    1511
Name: count, dtype: int64


In [59]:
df = df.drop_duplicates()

In [60]:
print(df['label'].value_counts())

label
0    3202
1    1407
Name: count, dtype: int64


In [61]:
df['dataset'] = 'fox-comment'

file_path = os.path.join('data', 'fox-comment.csv')
df.to_csv(file_path, index=False)

### 11 DIALOCONAN

In [75]:
query = """
SELECT text.text, label.label_value as label, text.source_id
FROM text
JOIN label ON label.text_id = text.text_id
WHERE text.dataset_id = 11 AND label.label_name = 'type';
"""
df = pd.read_sql_query(query, conn)

In [72]:
print(len(df))
print(df.describe())
print(df.head(10))

39162
       source_id
count  12.000000
mean    9.500000
std     1.167748
min     8.000000
25%     8.750000
50%     9.500000
75%    10.250000
max    11.000000
                                                text     label  source_id
0  We’ve just imported 20k ticking time bombs fro...  notgiven        8.0
1  Surely people who also risked their lives to h...  notgiven        9.0
2  Our own must always come first. We have at lea...  notgiven       10.0
3  We also have at least 1 million empty homes. T...  notgiven       11.0
4  Our soldiers are left to rot on our streets wh...  notgiven        NaN
5  Neither of those statements is true. I saw sca...      none        NaN
6  I’m a patriot, I care about the future of our ...  notgiven        NaN
7  Can I suggest that greatness can be found in c...      none        NaN
8  Only 1 day more before the end of Black Histor...  notgiven        NaN
9                            What’s racist about it?  notgiven        NaN


In [68]:
print(df['label'].value_counts())

label
none                    10982
HS                       8314
CN                       8311
derogation               6840
animosity                1782
notgiven                 1589
threatening               443
dehumanization            398
dehumanizinglanguage      233
threateninglanguage       203
support                    67
Name: count, dtype: int64


In [78]:
df = df.dropna(subset=['source_id'])

In [79]:
print(df['label'].value_counts())

label
notgiven               4
HS                     2
CN                     2
threateninglanguage    2
none                   1
derogation             1
Name: count, dtype: int64


### 1 OLID


In [84]:
query = """
SELECT text.text, label.label_value as label
FROM text
JOIN label ON label.text_id = text.text_id
WHERE text.dataset_id = 2 AND label.label_name = 'subtask_a';
"""
df = pd.read_sql_query(query, conn)

In [85]:
print(len(df))
print(df.describe())
print(df.head(10))

13768
                                                     text  label
count                                               13768  13768
unique                                              13207      2
top     @USER Looks Like The Jokes On Liberals Again. ...    NOT
freq                                                    7   9185
                                                text label
0  @USER You said it. They're trying to prove tha...   OFF
1  @USER @USER Congress tried Secret Shah Bano st...   OFF
2  @USER @USER Cowards in masks. Antifa are the t...   NOT
3                    @USER @USER @USER Antifa lives!   OFF
4  @USER @USER She’s a class act isn’t she. Kim t...   NOT
5  Political Unmasking of Team Trump Must Be Pros...   NOT
6  #MOGG: #JacobReesMogg calls for @USER to #Chuc...   NOT
7  @USER More misleading nonsense. It’s like you ...   NOT
8  .@USER yet again siding with the .@USER and st...   NOT
9  @USER @USER @USER @USER @USER @USER @USER @USE...   NOT


In [86]:
df['label'] = df['label'].replace({'NOT': 0, 'OFF':1})

In [87]:
print(df['label'].value_counts())

label
0    9185
1    4583
Name: count, dtype: int64


In [89]:
df = df.drop_duplicates()
print(df['label'].value_counts())

label
0    8943
1    4520
Name: count, dtype: int64


In [90]:
df['dataset'] = 'OLID19'

file_path = os.path.join('data', 'OLID19.csv')
df.to_csv(file_path, index=False)